In [1]:
import lightkurve as lk
path = os.path.dirname(os.getcwd())

sys.path.append(path)
from kepler_apertures import KeplerFFI
from kepler_apertures.utils import get_gaia_sources

import astropy.units as u
import numpy as np
from pyia import GaiaData
from astroquery.gaia import Gaia, GaiaClass, TapPlus

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [2]:
get_gaia_sources.cache_clear()

In [3]:
ras = [281.97159801]
decs = [44.29987168]
rads = [1.1977325732964266]
epoch = 2009.3120406806293

In [4]:
Gaia.login(user="jmarti06", password="Antares24.10")

OK


In [5]:
wheres = [
        f"""1=CONTAINS(
                  POINT('ICRS',ra,dec),
                  CIRCLE('ICRS',{ra},{dec},{rad}))"""
        for ra, dec, rad in zip(ras, decs, rads)
    ]

where = """\n\tOR """.join(wheres)

In [6]:
query_txt = f"""SELECT designation,
                    coord1(prop) AS ra, ra_error, coord2(prop) AS dec, dec_error, parallax,
                    parallax_error, pmra, pmra_error, pmdec, pmdec_error, dr2_radial_velocity, dr2_radial_velocity_error,
                    ruwe, phot_g_n_obs, phot_g_mean_flux,
                    phot_g_mean_flux_error, phot_g_mean_mag,
                    phot_bp_n_obs, phot_bp_mean_flux, phot_bp_mean_flux_error, phot_bp_mean_mag, phot_rp_n_obs,
                    phot_rp_mean_flux, phot_rp_mean_flux_error,
                    phot_rp_mean_mag FROM (
             SELECT *,
             EPOCH_PROP_POS(ra, dec, parallax, pmra, pmdec, 0, ref_epoch, {epoch}) AS prop
             FROM gaiaedr3.gaia_source
             WHERE {where}
            )  AS subquery
            WHERE phot_g_mean_mag<=20
            """

In [10]:
import socket
socket.setdefaulttimeout(1000)


In [11]:
job = Gaia.launch_job_async(query_txt, verbose=True)
r = job.get_results()

Launched query: 'SELECT designation,
                    coord1(prop) AS ra, ra_error, coord2(prop) AS dec, dec_error, parallax,
                    parallax_error, pmra, pmra_error, pmdec, pmdec_error, dr2_radial_velocity, dr2_radial_velocity_error,
                    ruwe, phot_g_n_obs, phot_g_mean_flux,
                    phot_g_mean_flux_error, phot_g_mean_mag,
                    phot_bp_n_obs, phot_bp_mean_flux, phot_bp_mean_flux_error, phot_bp_mean_mag, phot_rp_n_obs,
                    phot_rp_mean_flux, phot_rp_mean_flux_error,
                    phot_rp_mean_mag FROM (
             SELECT *,
             EPOCH_PROP_POS(ra, dec, parallax, pmra, pmdec, 0, ref_epoch, 2009.3120406806293) AS prop
             FROM gaiaedr3.gaia_source
             WHERE 1=CONTAINS(
                  POINT('ICRS',ra,dec),
                  CIRCLE('ICRS',281.97159801,44.29987168,1.1977325732964266))
            )  AS subquery
            WHERE phot_g_mean_mag<=20
            '
------>https
host 

TimeoutError: [Errno 60] Operation timed out

In [ ]:
r